# <span style="color:blue">Title: Gene Ontology Enrichmen Analysis (GOEA) with </span><span style="color:red">Python's goatools.</span>

# <span style="color:blue">Gene Ontology Enrichment Analysis (GOEA) Python Tutorial<span style="color:blue">.</span>
* ## References
    * ## Tang H. goatools: Python library to handle Gene Ontology (GO) terms.
        * ## Process over- and under-representation of certain GO terms, based on Fisher's exact test. With numerous multiple correction routines including locally implemented routines for Bonferroni, Sidak, Holm, and false discovery rate. Also included are multiple test corrections from statsmodels: FDR Benjamini/Hochberg, FDR Benjamini/Yekutieli, Holm-Sidak, Simes-Hochberg, Hommel, FDR 2-stage Benjamini-Hochberg, FDR 2-stage Benjamini-Krieger-Yekutieli, FDR adaptive Gavrilov-Benjamini-Sarkar, Bonferroni, Sidak, and Holm.

    * ## Gene ontology python tutorial. 2022 Feb 23. Kaggle.com. [accessed 2023 Oct 22]. https://www.kaggle.com/code/alexandervc/gene-ontology-python-tutorial.
    
    * ## Sanbomics. 2022 May 15. How to do gene ontology analysis in python. [accessed 2023 Oct 22]. https://www.youtube.com/watch?v=ONiWugVEf2s.

# <span style="color:blue">Competency</span>
* ## Become proficient in performing gene ontology enrichment analysis for marker genes from HuBMAP's scRNA-seq.

# <span style="color:blue">Objectives</span>
* ## Define the list of marker genes for a cluster of interest from a HuBMAP data analysis.
* ## Perform Gene Ontology Enrichment Analysis (GOEA) for a cluster of interest from a HuBMAP data analysis.

# <span style="color:blue">Install the required </span><span style="color:red">Python libraries</span><span style="color:blue"> to conduct the </span><span style="color:red">gene ontology analysis</span><span style="color:blue">.</span>

In [ ]:
# Install goatools
!pip install goatools -q

# Install pandas
!pip install pandas -q

# <span style="color:blue">Load required </span><span style="color:red">Python libraries</span><span style="color:blue">.</span>

In [ ]:
# goattols: A Python library for Gene Ontology analyses

# Import the OBO parser from goatools
#     Open Biomedical Ontologies (OBO)
from goatools import obo_parser

#import os
import os

# import pandas
import pandas as pd

# <span style="color:blue">Create a list for the marker genes from the scRNA-seq data analysis obtained with Python's library scanpy.</span>

In [ ]:
marker_genes_c0_vs_rest = ['RPS3A', 'RPS12', 'EEF1A1', 'RPL10', 'RPS2', 'RPL3', 'RPL18A', 'RPL32', 'RPL13', 'RPS4X', 'RPS3',
                           'RPS19', 'RPS8', 'RPS15A', 'RPS6', 'RPS13', 'RPS27A', 'RPS24', 'RPL19', 'RPL11', 'RPL34', 'RPS18',
                           'RPS18', 'RPL29', 'RPS10']

# <span style="color:blue">Obtain the background gene set from NCBI.</span>
* ## Connect to https://www.ncbi.nlm.nih.gov/gene/

* ## Copy and paste the text in blue into the NCBI gene search field <span style="color:red">"9606"[Taxonomy ID] AND alive[property] AND genetype protein coding[Properties]</span>
    * ## <span style="color:blue">9606</span> is the taxonomy ID for <span style="color:blue">Homo sapiens</span>.
    * ## <span style="color:blue">alive[property]</span> is used to obtain the records that are current and primary (i.e., not secondary or discontinued). 
* ## Download NCBI results by send it into a tabular (text) file.
* ## Rename the file from <span style="color:red">gene_result.txt</span> to <span style="color:blue">gene_result_human.txt</span>.
* ## Move the file from download into your working directory.

# Now we need to locate the Python script <span style="color:blue">ncbi_gene_results_to_python.py</span> that was installed in the Python directory.
* ## This information is provided by the PSC's Software Specialist Iván Cao-Berg.
* ## Edit if necessary the path to the file <span style="color:blue">ncbi_gene_results_to_python.py</span> in the following line to convert NCBI Gene tabular file to a Python module.
    * ## !python <span style="color:red">/jet/home/$(whoami)/.local/lib/python3.9/site-packages/goatools/cli/ncbi_gene_results_to_python.py</span> <span style="color:blue">-o genes_ncbi_homo_sapiens_proteincoding.py</span> <span style="color:green">gene_result_human.txt</span> -o means output file.

In [ ]:
!python /jet/home/$(whoami)/.local/lib/python3.9/site-packages/goatools/cli/ncbi_gene_results_to_python.py -o genes_ncbi_homo_sapiens_proteincoding.py gene_result_human.txt

# Load <span style="color:blue">Background Gene Set</span> for <span style="color:blue">Human</span>
* ## Now we import NCBI data from the Python module <span style="color:red">genes_ncbi_homo_sapiens_proteincoding.py</span> that was created in the previous step and we will call it <span style="color:blue">Gene2IDnt_human</span>.
* ## All of the background genes for our HuBMAP dataset code for human proteins.

In [ ]:
from genes_ncbi_homo_sapiens_proteincoding import GENEID2NT as GeneID2nt_human

# Display the length of the GeneID2nt_human variable.
print('Length of GeneID2nt_human =', len(GeneID2nt_human))

# <span style="color:blue">Load required </span><span style="color:red">goatools packages</span><span style="color:blue">.</span>

In [ ]:
from goatools.base import download_go_basic_obo
from goatools.base import download_ncbi_associations
from goatools.obo_parser import GODag
from goatools.anno.genetogo_reader import Gene2GoReader
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS

# <span style="color:blue">Download Ontologies</span>

In [ ]:
# Download Ontologies
#     Open Biomedical Ontologies (OBO)
obo_file_name = download_go_basic_obo()

# <span style="color:blue">Load Ontologies</span>

In [ ]:
# Load Ontologies
#   Open Biomedical Ontologies (OBO)
#   Gene Ontology Directed Acyclic Graphs
obo_dag = GODag("go-basic.obo")

# <span style="color:blue">Display Ontologies, a dictionary of GO terms and its metadata.</span>

In [ ]:
# Display obo_dag
obo_dag

# <span style="color:blue">Download Associations</span>
* ## There are several species in the NCBI gene2go dataset.
* ## But we will choose Human.

In [ ]:
# Download Associations
input_file_gene2go = download_ncbi_associations()

# <span style="color:blue">Load Associations</span>
* ## <span style="color:blue">9606</span> is the taxonomy ID for <span style="color:blue">Homo sapiens</span>

In [ ]:
# Read NCBI annotation file, "gene2go" for human.
#    9606 is the taxonomy ID for Homo sapiens
objanno = Gene2GoReader(input_file_gene2go, taxids=[9606])

# namespace2association:
#    namespace:
#        BP: Biological Process
#        CC: Cellular Component
#        MF: Molecular Function
#
#    assocation (dictionary):
#        key: NCBI GeneID
#        value: A set of GO IDs associated with that gene
ns2assoc = objanno.get_ns2assc()

# <span style="color:blue">Display Associations</span>

In [ ]:
# Display Associations for Biological Process
ns2assoc['BP']

In [ ]:
# Display Associations for Cellular Component
ns2assoc['CC']

In [ ]:
# Display Associations for Molecular Function
ns2assoc['MF']

# <span style="color:blue">Create a dictionary called mapper with the gene symbols as the key and the gene ID number as the value.</span>
* ## We need this dictionary because our scRNA-seq data uses the gene HUGO symbols.

In [ ]:
# Initialize the variable mapper as an emtpy dictionary.
mapper = {}

# Traverse the GeneID2nt_human dictionary.
for key in GeneID2nt_human:
    mapper[GeneID2nt_human[key].Symbol] = GeneID2nt_human[key].GeneID

# Create another dictionary called inv_map with the Gene ID number as the key and the gene symbol as the value.
inv_map = {v: k for k, v in mapper.items()}

# <span style="color:blue">Display the dictionary mapper.</span>

In [ ]:
# Display the dictionary mapper.
mapper

# <span style="color:blue">Display the dictionary inv_map.</span>

In [ ]:
# Display the dictionary inv_map.
inv_map

# Display the information for the <span style="color:blue">Gene ID 1353</span> from the variable <span style="color:blue">GeneID2nt_human</span>.

In [ ]:
GeneID2nt_human[1353]

# <span style="color:blue">Run the function GOEnrichmentStudyNS to initialize the GO object.</span>
* ## We will use the GO Enrichment Analysis objetc (variable goea_obj) for running enrichment studies.
* ## The Gene Ontology Enrichment Analysis Object (variable goea_obj) contains the Ontologies, Associations, and background genes.

In [ ]:
# Initialize the goea_obj by running the GOEnrichmentStudyNS function.
goea_obj = GOEnrichmentStudyNS(
        GeneID2nt_human.keys(), # List of human protein-coding genes.
        ns2assoc, # Gene ID / GO Associations
        obo_dag, # Ontologies
        propagate_counts = False, # Use the annotations in their original form with no modifications.
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method

# <span style="color:blue">Create a list of all GO terms combined from Biological Process, Cellular Component, and Molecular Function.</span>

In [ ]:
# Create a list of all the GO terms.
GO_terms_list = []

# Travers the GO terms for Biological Process.
temp_bp = goea_obj.ns2objgoea['BP'].assoc
for item in temp_bp:
    GO_terms_list += temp_bp[item]
    
# Travers the GO terms for Cellular Component.
temp_cc = goea_obj.ns2objgoea['CC'].assoc
for item in temp_cc:
    GO_terms_list += temp_cc[item]

# Traverse the GO terms for Molecular Function.
temp_mf = goea_obj.ns2objgoea['MF'].assoc
for item in temp_mf:
    GO_terms_list += temp_mf[item]

# <span style="color:blue">Display the count of genes that are associated to a specific GO term.</span>
* ## As an example we will use <span style="color:blue">GO Molecular Function GO:0002020 Protease Binding</span>.

In [ ]:
# GO Molecular Function
#    GO:0002020 Protease Binding
#    This GO term has 109 genes of this GO term in the list.
GO_terms_list.count('GO:0002020')

# <span style="color:blue">Map the genes from our list of Tagert Genes (variable ) using the dictionary mapper that we created previously.</span>

In [ ]:
# Create an empty list for the mapped genes (variable mapped_genes).
mapped_genes = []
    
 # Traverse the list of Target Genes (variable target_genes).
for gene in marker_genes_c0_vs_rest:
    # The try statement is completed and the except clause is skipped if there are no errors duirng the mapping process.
    try:
        # Append the dictionary value for the gene being processed.
        mapped_genes.append(mapper[gene])
    # If the gene can not be mapped, display the gene name and pass onto the next gene in the list.
    except:
        # Display the name of the gene that could not be mapped.
        print('\n\t%s was not mapped.\n' % (gene))
    
# Display the amount of genes that were mapped.
print(f'\nMapped genes: {len(mapped_genes)}')

# Display the genes that were mapped.
print('\n', mapped_genes)

# <span style="color:blue">Perform the Gene Ontology Enrichment Analysis (GOEA)</span>

In [ ]:
# Function to perform GOEA on the list of gene symbols.
#
# Arguments
#    test_genes
#
def gene_ontology_enrichment_analysis(mapped_genes):
    # Display the list of target_genes.
    print(f'\nTarget genes: {len(mapped_genes)}', '\n')
        
    # Execute the run_study function to perform the Gene Ontology Enrichment Analysis (GOEA).
    goea_results_all = goea_obj.run_study(mapped_genes)
    
    # Filter the results based on the corrected p-value to keep only the significant results.
    #    'p_' means "p-value".
    #    'fdr_bh' is the multiple-test method we are currently using False Discovery Rate (FDR) Benjamini-Hochberg.
    goea_results_sig = [result for result in goea_results_all if result.p_fdr_bh < 0.05]
    
    # Create a visually readable data frame from the significant results.
    GO = pd.DataFrame(list(map(lambda x: [x.GO, x.goterm.name, x.goterm.namespace, x.p_uncorrected, x.p_fdr_bh,\
                   x.ratio_in_study[0], x.ratio_in_study[1], GO_terms_list.count(x.GO), list(map(lambda y: inv_map[y], x.study_items)),\
                   ], goea_results_sig)), columns = ['GO', 'term', 'class', 'p', 'p_corr', 'n_genes',\
                                                    'n_study', 'n_go', 'study_genes'])

    # Select the GO result with a number of genes greater than 1.
    GO = GO[GO.n_genes > 1]
    return GO

In [ ]:
# Perform the Gene Ontology Enrichment Analysis
goea_data_frame = gene_ontology_enrichment_analysis(mapped_genes)

# <span style="color:blue">Display the data frame (variable) with the GOEA results sorted.</span>

In [ ]:
# Sort first by p-value (p_value) and then by p_corr.
goea_df_sorted = goea_data_frame.sort_values(['p', 'p_corr'], ascending = [True, True])

# Display the data frame for the Gene Ontology Enrichment Analysis.
goea_df_sorted

# <span style="color:blue">Export the results to a Comma Separated Value (CSV) file.</span>

In [ ]:
# Export the results to a CSV file.
goea_df_sorted.to_csv('goae_sorted_goatools_cluster_0.csv')

# <span style="color:blue">Compute the genes ratio from the number of genes obtained divided by the number of genes in the GO term.</span>

In [ ]:
# Compute the genes ratio from the number of genes obtained divided by the number of genes in the GO term.
goea_df_sorted['genes_ratio'] = goea_df_sorted.n_genes/goea_df_sorted.n_go

In [ ]:
# Display the table.
goea_df_sorted

# <span style="color:blue">Load Python libraries to create a plot for the results.</span>

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import seaborn as sns
import textwrap

# <span style="color:blue">Select the top 10 results.</span>

In [ ]:
goea_df_sorted_top_10 = goea_df_sorted[0:10]

# <span style="color:blue">Create a color map from the minimum corrected p-value to the maximum corrected p-value.</span>

In [ ]:
# Display the minimum and maximum FDR value for Function.
print('\n\tMinimum FDR =', goea_df_sorted_top_10.p_corr.min())

print('\n\tMaximum FDR =', goea_df_sorted_top_10.p_corr.max())

# <span style="color:blue">Generate the barplot with the significant results from our Gene Ontology Enrichment Analysis.</span>

In [ ]:
norm = mpl.colors.Normalize(vmin = goea_df_sorted_top_10.p_corr.min(),
                            vmax = goea_df_sorted_top_10.p_corr.max())

mapper = cm.ScalarMappable(norm = norm, cmap = cm.bwr_r)

mapper.set_array([])

plt.figure(figsize = (6,10))

ax = sns.barplot(data = goea_df_sorted_top_10,
                 x = 'genes_ratio',
                 y = 'term',
                 palette = mapper.to_rgba(goea_df_sorted_top_10.p_corr.values))

# Add the values to the bars
ax.bar_label(ax.containers[0])

ax.figure.colorbar(mapper, ax=ax)

ax.set_yticklabels([textwrap.fill(go_term, 22) for go_term in goea_df_sorted_top_10['term']])

# Add Plot Title
ax.set_title('Barplot for the Gene Ontology terms for Cluster 0',
             loc ='center', fontweight ='bold', fontsize = 10)

# Adjust the scale of the x-axis.
ax.set_xlim(0.0, 0.4)

# Add the x-axis label 
plt.xlabel('Genes ratio', fontweight ='bold', fontsize = 10) 

# Add the x-axis label 
plt.ylabel('Gene Ontology Term', fontweight ='bold', fontsize = 10) 

plt.show()

# <span style="color:blue">Obtain individual results by Function, Process, and Component and select top 10 results.</span>

In [ ]:
# Class = Molecular Function
enrich_goea_df_sig_function = goea_df_sorted[(goea_df_sorted['class'] == 'molecular_function')]

# Select the top 10 results for Molecular Function.
enrich_goea_df_sig_function_top10 = enrich_goea_df_sig_function[0:10]

# Class = Biological Process
enrich_goea_df_sig_process = goea_df_sorted[(goea_df_sorted['class'] == 'biological_process')]

# Select the top 10 results for Biological Process.
enrich_goea_df_sig_process_top10 = enrich_goea_df_sig_process[0:10]

# Category = Cellular Component
enrich_goea_df_sig_component = goea_df_sorted[(goea_df_sorted['class'] == 'cellular_component')]

# Select the top 10 results for Cellular Component.
enrich_goea_df_sig_component_top10 = enrich_goea_df_sig_component[0:10]

# <span style="color:blue">Generate the barplot with the significant results from our Gene Ontology Enrichment Analysis for Molecular Function.</span>

## <span style="color:blue">Display </span><span style="color:red">top 10 significant gene ontology results</span><span style="color:blue"> for the Molecular Function.</span>

In [ ]:
# Display the top 10 significant gene ontology results for the Molecular Function.
enrich_goea_df_sig_function_top10

## <span style="color:blue">Barplot for the Gene Ontology results for Molecular Function with a color map legend from the minimum corrected p-value to the maximum corrected p-value.</span>

In [ ]:
# Display the minimum and maximum FDR value for Function.
print('Category = Molecular Function')
print('\n\tMinimum FDR =', enrich_goea_df_sig_function_top10['p_corr'].min())

print('\n\tMaximum FDR =', enrich_goea_df_sig_function_top10['p_corr'].max())

In [ ]:
norm = mpl.colors.Normalize(vmin = enrich_goea_df_sig_function_top10['p_corr'].min(),
                            vmax = enrich_goea_df_sig_function_top10['p_corr'].max())

mapper = cm.ScalarMappable(norm = norm, cmap = cm.bwr_r)

fig, ax = plt.subplots(figsize = (5, 5))

ax = sns.barplot(data = enrich_goea_df_sig_function_top10,
                 x = enrich_goea_df_sig_function_top10['genes_ratio'],
                 y = enrich_goea_df_sig_function_top10['term'],
                 palette = mapper.to_rgba(enrich_goea_df_sig_function_top10['p_corr']),
                 dodge=False)

# Add the values to the bars
ax.bar_label(ax.containers[0])

ax.figure.colorbar(mapper, ax=ax)

ax.set_yticklabels([textwrap.fill(go_term, 22) for go_term in enrich_goea_df_sig_function_top10['term']])

# Add Plot Title
ax.set_title('Barplot for Molecular Function GO terms\nfor Cluster 0',
             loc ='center', fontweight ='bold', fontsize = 10)

# Adjust the scale of the x-axis.
ax.set_xlim(0.0, 0.2)

# Add minor ticks.
ax.xaxis.set_minor_locator(AutoMinorLocator())

# Add the x-axis label 
plt.xlabel('Genes ratio', fontweight ='bold', fontsize = 10) 

# Add the x-axis label 
plt.ylabel('Gene Ontology Term', fontweight ='bold', fontsize = 10)

plt.show()

# <span style="color:blue">Generate the barplot with the significant results from our Gene Ontology Enrichment Analysis for Biological Process.</span>

## <span style="color:blue">Display </span><span style="color:red">top 10 significant gene ontology results</span><span style="color:blue"> for the Biological Process.</span>

In [ ]:
# Display the top 10 significant gene ontology results for the Molecular Function.
enrich_goea_df_sig_process_top10

## <span style="color:blue">Barplot for the Gene Ontology results for Biological Process with a color map legend from the minimum corrected p-value to the maximum corrected p-value.</span>

In [ ]:
# Display the minimum and maximum FDR value for Function.
print('Category = Biological Process')
print('\n\tMinimum FDR =', enrich_goea_df_sig_process_top10['p_corr'].min())

print('\n\tMaximum FDR =', enrich_goea_df_sig_process_top10['p_corr'].max())

In [ ]:
norm = mpl.colors.Normalize(vmin = enrich_goea_df_sig_process_top10['p_corr'].min(),
                            vmax = enrich_goea_df_sig_process_top10['p_corr'].max())

mapper = cm.ScalarMappable(norm = norm, cmap = cm.bwr_r)

fig, ax = plt.subplots(figsize = (5, 5))

ax = sns.barplot(data = enrich_goea_df_sig_process_top10,
                 x = enrich_goea_df_sig_process_top10['genes_ratio'],
                 y = enrich_goea_df_sig_process_top10['term'],
                 palette = mapper.to_rgba(enrich_goea_df_sig_process_top10['p_corr']),
                 dodge=False)

# Add the values to the bars
ax.bar_label(ax.containers[0])

ax.figure.colorbar(mapper, ax=ax)

ax.set_yticklabels([textwrap.fill(go_term, 22) for go_term in enrich_goea_df_sig_process_top10['term']])

# Add Plot Title
ax.set_title('Barplot for Biological Process GO terms\nfor Cluster 0',
             loc ='center', fontweight ='bold', fontsize = 10)

# Adjust the scale of the x-axis.
ax.set_xlim(0.0, 0.35)

# Add minor ticks.
ax.xaxis.set_minor_locator(AutoMinorLocator())

# Add the x-axis label 
plt.xlabel('Genes ratio', fontweight ='bold', fontsize = 10) 

# Add the x-axis label 
plt.ylabel('Gene Ontology Term', fontweight ='bold', fontsize = 10)

plt.show()

# <span style="color:blue">Generate the barplot with the significant results from our Gene Ontology Enrichment Analysis for Cellular Component.</span>

## <span style="color:blue">Display the </span><span style="color:red">top 10 significant gene ontology results</span><span style="color:blue"> for the Cellular Component.</span>

In [ ]:
# Display the top 10 significant gene ontology results for the Molecular Function.
enrich_goea_df_sig_component_top10

## <span style="color:blue">Barplot for the Gene Ontology results for Cellular Component with a color map legend from the minimum corrected p-value to the maximum corrected p-value.</span>

In [ ]:
# Display the minimum and maximum FDR value for Function.
print('Category = Cellular Component')
print('\n\tMinimum FDR =', enrich_goea_df_sig_component_top10['p_corr'].min())

print('\n\tMaximum FDR =', enrich_goea_df_sig_component_top10['p_corr'].max())

In [ ]:
norm = mpl.colors.Normalize(vmin = enrich_goea_df_sig_component_top10['p_corr'].min(),
                            vmax = enrich_goea_df_sig_component_top10['p_corr'].max())

mapper = cm.ScalarMappable(norm = norm, cmap = cm.bwr_r)

fig, ax = plt.subplots(figsize = (5, 5))

ax = sns.barplot(data = enrich_goea_df_sig_component_top10,
                 x = enrich_goea_df_sig_component_top10['genes_ratio'],
                 y = enrich_goea_df_sig_component_top10['term'],
                 palette = mapper.to_rgba(enrich_goea_df_sig_component_top10['p_corr']),
                 dodge=False)

# Add the values to the bars
ax.bar_label(ax.containers[0])

ax.figure.colorbar(mapper, ax=ax)

ax.set_yticklabels([textwrap.fill(go_term, 22) for go_term in enrich_goea_df_sig_component_top10['term']])

# Add Plot Title
ax.set_title('Barplot for Cellular Component GO terms\nfor Cluster 0',
             loc ='center', fontweight ='bold', fontsize = 10)

# Adjust the scale of the x-axis.
ax.set_xlim(0.0, 0.5)

# Add minor ticks.
ax.xaxis.set_minor_locator(AutoMinorLocator())

# Add the x-axis label 
plt.xlabel('Genes ratio', fontweight ='bold', fontsize = 10) 

# Add the x-axis label 
plt.ylabel('Gene Ontology Term', fontweight ='bold', fontsize = 10)

plt.show()